In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import tensorflow as tf

In [2]:
load_dotenv()

data_path = os.getenv("FILES_PATH")
images_path = os.path.join(data_path, "PNG", "collaborative_filtering")
os.makedirs(images_path, exist_ok=True)

def save_fig(fig_id, tight_layout=True, extension="png", resolution=300):  # Función para guardar las figuras que se vayan generando
    img_path = os.path.join(IMAGES_PATH, fig_id + "." + extension)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(img_path, format=extension, dpi=resolution)

In [3]:
from matplotlib import pyplot as plt

# Configuración de parámetros de matplotlib

plt.rc("font", size=14)
plt.rc("axes", labelsize=14, titlesize=14)
plt.rc("legend", fontsize=14)
plt.rc("xtick", labelsize=10)
plt.rc("ytick", labelsize=10)

In [4]:
df = pd.read_csv(os.path.join(data_path, "CSV", "cleaned_ratings.csv"), low_memory=False)

# Collaborative Filtering

El filtro colaborativo es una técnica utilizada para la recomendación de ítems, basada en las valoraciones y otros parámetros como _likes_ que los usuarios dan a los ítems. De esta forma, se realizan recomendaciones basadas en lo que otros usuarios han comprado o han visto. Al igual que en el filtro de contenido lo que hacíamos era computar la similaridad entre metadatos o sinopsis de las películas, en este caso vamos a utilizar la similaridad entre los usuarios, según las valoraciones que han dado a las películas.

Antes de nada, vamos a comprobar cuán dispersa es una matriz de usuarios, películas.

In [5]:
df_shuffled = df.drop(columns=["timestamp"]).sample(frac=1, random_state=42)
idx = int(0.9 * len(df_shuffled))
df_train = df_shuffled[:idx]
df_test = df_shuffled[idx:]

In [6]:
df_pivot = df_train.pivot_table(index="userId", columns="movieId", values="rating")
print(f"Shape of the new table {(df_pivot.shape)}")
df_pivot.sample(3)

/var/folders/gz/bntb_6c1617cvy8kh1mz_mnw0000gn/T/ipykernel_3627/2304156484.py:1: PerformanceWarning: The following operation may generate 11966947627 cells in the resulting pandas object.
  df_pivot = df_train.pivot_table(index="userId", columns="movieId", values="rating")


KeyboardInterrupt: 

Nuestra matriz es demasiado dispersa, por lo que utilizaremos el _DataFrame_ importado directamente del CSV. 

## Descomposición Matricial

Vamos a factorizar nuestro _dataset_ matricial en un producto de matrices: una matriz de usuarios y una matriz de items (películas en nuestro caso). Cada matriz contendrá parámetros asociados a cada película y cada usuario, como si hiciésemos una regresión lineal por película y usuario. Para entrenar este modelo, utilizaremos el método de descenso de gradiente para que el algoritmo encuentre las variables latentes que representen las matrices descompuestas.

In [ ]:
user_mapper = {usr_id: i for i, usr_id in enumerate(df_train["userId"].unique())}
movie_mapper = {mov_id: i for i, mov_id in enumerate(df_train["movieId"].unique())}

In [ ]:
user_train, user_test = df_train["userId"].map(user_mapper), df_test["userId"].map(user_mapper)
movie_train, movie_test = df_train["movieId"].map(movie_mapper), df_test["movieId"].map(movie_mapper)

In [ ]:
user_emb = len(user_mapper)
movie_emb = len(movie_mapper)
embedding_dim = 10

In [ ]:
user_input = tf.keras.layers.Input(shape=(1,), name="user_in")
movie_input = tf.keras.layers.Input(shape=(1,), name="movie_in")

user_embeddings = tf.keras.layers.Embedding(output_dim=embedding_dim,
                                           input_dim=user_emb,
                                           input_length=1,
                                           name="user_embedding_layer")(user_input)

movie_embeddings = tf.keras.layers.Embedding(output_dim=embedding_dim,
                                             input_dim=movie_emb,
                                             input_length=1,
                                             name="movie_embedding_layer")(movie_input)

user_vector = tf.keras.layers.Reshape([embedding_dim])(user_embeddings)
movie_vector = tf.keras.layers.Reshape([embedding_dim])(movie_embeddings)

y = tf.keras.layers.Dot(1, normalize=False)([user_vector, movie_vector])

model = tf.keras.Model(inputs=[user_input, movie_input], outputs=y)
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam())

model.fit([user_train, movie_train],
          df_train["rating"],
          batch_size=64, 
          epochs=5,
          validation_split=0.1,
          shuffle=True)

In [ ]:
y_hat = model.predict([user_test, movie_test])
y_true = df_test["rating"].values

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_hat, y_true))
print(f"Keras Matrix Factorization RMSE: {rmse:.5f}")

In [ ]:
df_test["predicted"] = y_hat.ravel()
df_test.head()

In [ ]:
models_path = os.path.join(data_path, "models")
collaborative_path = os.path.join(models_path, "collaborative_filtering")
if not os.path.exists(collaborative_path):
    os.mkdir(collaborative_path)
model.save(os.path.join(collaborative_path, "collaborative_matrix_decomposition.keras"))

## Deep Learning

Otra forma de crear un recomendador de filtro colaborativo es tener dos redes neuronales: una para usuario y otra para items; minimizando una función de coste que nos permita medir la distancia entre los vectores codificados de cada red (típicamente la norma $L_{2}$). También puede realizarse algo similar a lo que hicimos en el apartado anterior, pasar los usuarios y películas por capas separadas de Embedding y concatenar las salidas para llevarlas a una red neuronal común. Debido a que el volumen de datos es relativamente grande, vamos a utilizar el segundo acercamiento, ya que tardará menos en ser entrenado.

In [ ]:
user_emb_dim = 20
movie_emb_dim = 20

user_input = tf.keras.layers.Input(shape=(1,), name="user_in")
movie_input = tf.keras.layers.Input(shape=(1,), name="movie_in")


user_embeddings = tf.keras.layers.Embedding(output_dim=user_emb_dim, 
                           input_dim=user_emb,
                           input_length=1, 
                           name="user_embedding")(user_input)

movie_embeddings = tf.keras.layers.Embedding(output_dim=movie_emb_dim, 
                            input_dim=movie_emb,
                            input_length=1, 
                            name="movie_embedding")(movie_input)


user_vector = tf.keras.layers.Reshape([user_emb_dim])(user_embeddings)
movie_vector = tf.keras.layers.Reshape([movie_emb_dim])(movie_embeddings)
concat = tf.keras.layers.Concatenate()([user_vector, movie_vector])


dense1 = tf.keras.layers.Dense(units=128, activation="relu", kernel_initializer="he_normal")(concat)
dense2 = tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer="he_normal")(dense1)
y = tf.keras.layers.Dense(units=1, activation="linear")(dense2)


model = tf.keras.Model(inputs=[user_input, movie_input], outputs=y)
model.compile(loss="mse", optimizer="adam")


model.fit([user_train, movie_train],
          df_train["rating"],
          batch_size=128, 
          epochs=8,
          validation_split=0.1,
          shuffle=True)

In [ ]:
y_hat = model.predict([user_test, movie_test])
y_true = df_test["rating"].values

In [ ]:
rmse = np.sqrt(mean_squared_error(y_hat, y_true))
print(f"Deep Learning RMSE: {rmse:.5f}")

In [ ]:
df_test["predicted_deep"] = y_hat.ravel()
df_test.head()

In [ ]:
model.save(os.path.join(collaborative_path, "collaborative_deep_learning.keras"))

En este _notebook_ hemos visto cómo realizar un sistema recomendador basado en el filtro colaborativo. Este sistema podría combinarse con uno basado en contenido para tener un recomendador híbrido. En un entorno de producción, este sistema colaborativo recomendaría a un nuevo usuario películas basadas en las valoraciones medias y a medida que el usuario consumiese películas, se las recomendaría en base a la similaridad con otros usuarios. 

En el filtro colaborativo, a parte de las valoraciones de los usuarios, podríamos realizar recomendaciones basadas en las valoraciones y la puntuación asociada a cada género de la película. Esta puntuación asociada al género podría inferirse con un algoritmo de filtro colaborativo como el que hemos realizado en este _notebook_.